In [ ]:
!nvidia-smi

Wed Jul  3 18:54:33 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.90.07              Driver Version: 550.90.07      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla P100-PCIE-16GB           Off |   00000000:00:04.0 Off |                    0 |
| N/A   34C    P0             25W /  250W |       0MiB /  16384MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
!sudo apt install build-essential python3-dev libopenblas-dev -y

In [ ]:
!pip install torch ninja

In [ ]:
!pip install -U git+https://github.com/richlukich/MinkowskiEngine -v --no-deps \
                          --config-settings="--install-option=--force_cuda" \
                          --config-settings="--install-option=--blas=openblas"

In [ ]:
!pip install faiss-gpu
!pip install loguru
!pip install fuzzywuzzy
!pip install paddlepaddle-gpu
!pip install paddleocr
!pip install numpy-quaternion
!pip install dvc
!pip install dvc_gdrive

In [ ]:
import torch
import MinkowskiEngine as ME

In [ ]:
DATA_ROOT = '/kaggle/input/itlp-campus-dataset/public'

In [ ]:
import pandas as pd

In [ ]:
train_df = pd.read_csv('/kaggle/input/itlp-campus-dataset/public/train.csv').drop(columns='Unnamed: 0')
train_df

,timestamp,track,front_cam_ts,back_cam_ts,tx,ty,tz,qx,qy,qz,qw,front_cam_text,back_cam_text
0,1676034260851770,2023-02-10-08-04-19-twilight,1676034260821540,1676034260865870,-23.839826,-17.655232,-1.474530,0.013051,0.028187,0.003870,0.999510,[],[]
1,1676034268718093,2023-02-10-08-04-19-twilight,1676034268687989,1676034268708533,-19.060815,-19.003152,-2.513494,0.017344,0.019243,-0.121944,0.992199,[],[]
2,1676034274365801,2023-02-10-08-04-19-twilight,1676034274404483,1676034274382407,-14.289875,-20.252643,-2.563523,0.019104,0.025107,0.103990,0.994078,[],['KE']
3,1676034280013503,2023-02-10-08-04-19-twilight,1676034280004376,1676034279987523,-10.200171,-17.323240,-2.689153,0.006686,0.016921,0.394677,0.918640,[],['KB']
4,1676034285358658,2023-02-10-08-04-19-twilight,1676034285387661,1676034285339092,-6.752502,-13.812644,-2.623646,0.003145,0.013536,0.374985,0.926927,[],['KE']
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1354,1678905610167111,2023-03-15-13-25-48-night,1678905610166555,1678905610148946,-208.692754,19.746243,-2.661155,0.010751,0.021335,0.389329,0.920789,['АРКТИКА'],[]
1355,1678905615814276,2023-03-15-13-25-48-night,1678905615783690,1678905615831108,-204.946252,23.201232,-2.763870,-0.001526,0.012262,0.270024,0.962774,['АРКТИКА'],[]
1356,1678905621059160,2023-03-15-13-25-48-night,1678905621067311,1678905621028555,-200.439570,25.320586,-2.827866,0.000658,0.011407,0.146494,0.989146,['АРКТИКА'],[]
1357,1678905626909006,2023-03-15-13-25-48-night,1678905626884315,1678905626895739,-195.479406,26.327213,-2.666329,0.007004,-0.019667,-0.149021,0.988614,['АВТОМАТ'],[]


In [ ]:
test_df = pd.read_csv('/kaggle/input/itlp-campus-dataset/public/test_with_rec_text.csv').drop(columns='Unnamed: 0')
test_df

,timestamp,track,front_cam_ts,back_cam_ts,tx,ty,tz,qx,qy,qz,qw,front_cam_text,back_cam_text
0,1676034260851770,2023-02-10-08-04-19-twilight,1676034260821540,1676034260865870,-23.839826,-17.655232,-1.474530,0.013051,0.028187,0.003870,0.999510,[],[]
1,1676034268718093,2023-02-10-08-04-19-twilight,1676034268687989,1676034268708533,-19.060815,-19.003152,-2.513494,0.017344,0.019243,-0.121944,0.992199,[],[]
2,1676034274365801,2023-02-10-08-04-19-twilight,1676034274404483,1676034274382407,-14.289875,-20.252643,-2.563523,0.019104,0.025107,0.103990,0.994078,[],['KE']
3,1676034280013503,2023-02-10-08-04-19-twilight,1676034280004376,1676034279987523,-10.200171,-17.323240,-2.689153,0.006686,0.016921,0.394677,0.918640,[],['KB']
4,1676034285358658,2023-02-10-08-04-19-twilight,1676034285387661,1676034285339092,-6.752502,-13.812644,-2.623646,0.003145,0.013536,0.374985,0.926927,[],['KE']
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1850,1678905610167111,2023-03-15-13-25-48-night,1678905610166555,1678905610148946,-208.692754,19.746243,-2.661155,0.010751,0.021335,0.389329,0.920789,['АРКТИКА'],[]
1851,1678905615814276,2023-03-15-13-25-48-night,1678905615783690,1678905615831108,-204.946252,23.201232,-2.763870,-0.001526,0.012262,0.270024,0.962774,['АРКТИКА'],[]
1852,1678905621059160,2023-03-15-13-25-48-night,1678905621067311,1678905621028555,-200.439570,25.320586,-2.827866,0.000658,0.011407,0.146494,0.989146,['АРКТИКА'],[]
1853,1678905626909006,2023-03-15-13-25-48-night,1678905626884315,1678905626895739,-195.479406,26.327213,-2.666329,0.007004,-0.019667,-0.149021,0.988614,['АВТОМАТ'],[]


In [ ]:
val_df = pd.read_csv('/kaggle/input/itlp-campus-dataset/public/val.csv').drop(columns='Unnamed: 0')
val_df

,timestamp,track,front_cam_ts,back_cam_ts,tx,ty,tz,qx,qy,qz,qw,front_cam_text,back_cam_text
0,1676035566075019,2023-02-10-08-04-19-twilight,1676035566108253,1676035566077315,80.724531,-71.308620,-0.873446,-0.000971,0.007920,0.327289,0.944891,['ФИЗТЕХ.БИО'],"['МФТИ', 'ЛАБОРАТОРНЫЙ КОРПУС', '1945', '1957']"
1,1676035571722667,2023-02-10-08-04-19-twilight,1676035571691405,1676035571749351,84.696672,-67.969684,-0.886570,-0.000097,0.017570,0.324952,0.945567,['ФИЗТЕХ.Б'],"['МФТИ', 'ЛАБОРАТОРНЫЙ КОРПУС', '1952']"
2,1676035577269474,2023-02-10-08-04-19-twilight,1676035577291312,1676035577284949,88.486198,-64.722221,-0.878774,-0.005348,0.022473,0.352585,0.935495,['РАДИОТЕХНИЧЕСКИЙ ФАКУЛЬТЕТ'],"['МФТИ', 'ЛАБОРАТОРНЫЙ КОРПУС']"
3,1676035582715506,2023-02-10-08-04-19-twilight,1676035582724463,1676035582747679,92.378366,-61.577601,-0.906462,-0.004888,0.020862,0.231267,0.972654,"['ФИЗТЕХ', 'РАДИОТЕХНИЧЕСКИЙ', 'КОРПУС']","['МФТИ', 'ЛАБОРАТОРНЫЙ КОРПУС']"
4,1676035588262317,2023-02-10-08-04-19-twilight,1676035588290997,1676035588289057,96.279563,-58.386627,-1.030017,-0.007974,0.017085,0.382182,0.923895,"['РАДИОТЕХНИЧЕСКИЙ', 'КОРПУС', 'P']","['МФТИ', 'ГОСУДАРСТВЕННЫЙ УНИВЕРСИТЕТ']"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
491,1678904283512023,2023-03-15-13-25-48-night,1678904283555720,1678904283534244,-155.078201,-194.151665,-1.393639,-0.024897,-0.018326,0.741418,0.670331,['ФИЗТ'],[]
492,1678904290067940,2023-03-15-13-25-48-night,1678904290056078,1678904290072465,-154.392377,-189.269135,-1.392422,-0.013549,0.009834,0.618619,0.785512,['ФИЗТЕХ. КВАНТ'],[]
493,1678904296724738,2023-03-15-13-25-48-night,1678904296739853,1678904296744134,-153.911107,-184.344878,-1.207735,-0.001541,0.003475,0.649004,0.760775,['Физтех'],[]
494,1678904302373180,2023-03-15-13-25-48-night,1678904302356838,1678904302348016,-153.259422,-179.365098,-1.463031,0.003824,0.010529,0.686563,0.726984,[],[]


In [ ]:
!git clone -b feat/itlp_outdoor https://github.com/alexmelekhin/open_place_recognition
%cd open_place_recognition
!pip install -e .
%cd ..

In [ ]:
!pip install git+https://github.com/alexmelekhin/open_place_recognition.git@feat/itlp_outdoor

In [ ]:
!pip install gdown

In [ ]:
from torch.utils.data import DataLoader

from opr.datasets.itlp_outdoor import ITLPCampusOutdoor
from opr.samplers import BatchSampler

In [ ]:
train_dataset = ITLPCampusOutdoor(
    DATA_ROOT,
    subset='train',
    sensors=('front_cam', 'back_cam'),
    load_semantics=True,
    #load_text_labels=True,
    mink_quantization_size=0.06,
)

val_dataset = ITLPCampusOutdoor(
    DATA_ROOT,
    subset='val',
    sensors=('front_cam', 'back_cam'),
    load_semantics=True,
    #load_text_labels=True,
    mink_quantization_size=0.06,
)

test_dataset = ITLPCampusOutdoor(
    DATA_ROOT,
    subset='test',
    sensors=('front_cam','back_cam'),
    load_semantics=True,
    #load_text_labels=True,
    mink_quantization_size=0.06,
)

In [ ]:
train_sampler = BatchSampler(
    train_dataset,
    batch_size=16,
    batch_size_limit=256,
    batch_expansion_rate=1.4,
    drop_last=True,
)

val_sampler = BatchSampler(
    val_dataset,
    batch_size=256,
    drop_last=True,
)

In [ ]:
train_dl = DataLoader(
    dataset=train_dataset,
    batch_sampler=train_sampler,
    collate_fn=train_dataset.collate_fn,
    pin_memory=True,
)

val_dl = DataLoader(
    dataset=val_dataset,
    batch_sampler=val_sampler,
    collate_fn=val_dataset.collate_fn,
    pin_memory=True,
)

test_dl = DataLoader(
    dataset=test_dataset,
    batch_size=256,
    collate_fn=test_dataset.collate_fn,
    pin_memory=True,
    drop_last=False,
)

In [ ]:
import torch
from opr.models.place_recognition.base import ImageModel
from opr.modules.feature_extractors import ConvNeXtTinyFeatureExtractor
from opr.modules import Concat, GeM

In [ ]:
import gc
torch.cuda.empty_cache()
gc.collect()

0

In [ ]:
feature_extractor = ConvNeXtTinyFeatureExtractor(
    in_channels=3,
    pretrained=True,
)

pooling = GeM()
descriptor_fusion_module = Concat()

model = ImageModel(
    backbone=feature_extractor,
    head=pooling,
    fusion=descriptor_fusion_module,
)

Downloading: "https://download.pytorch.org/models/convnext_tiny-983f1562.pth" to /root/.cache/torch/hub/checkpoints/convnext_tiny-983f1562.pth
100%|██████████| 109M/109M [00:00<00:00, 134MB/s]  


In [ ]:
IMAGE_LR = 0.001
HEAD_LR = None
FUSION_LR = None
WEIGHT_DECAY = 0.0001
WARMUP_STEPS = 90
T_MAX = 1800

In [ ]:
!pip install transformers

In [ ]:
from torch.optim import AdamW
from transformers import get_cosine_schedule_with_warmup

In [ ]:
params_list = []
if model.backbone is not None and IMAGE_LR is not None:
    params_list.append({"params": model.backbone.parameters(), "lr": IMAGE_LR})
if model.head is not None and HEAD_LR is not None:
    params_list.append({"params": model.head.parameters(), "lr": HEAD_LR})
if model.fusion is not None and FUSION_LR is not None:
    params_list.append({"params": model.fusion.parameters(), "lr": FUSION_LR})

In [ ]:
from opr.trainers.place_recognition import UnimodalPlaceRecognitionTrainer
from opr.losses import BatchHardTripletMarginLoss

In [ ]:
!mkdir checkpoints

In [ ]:
import wandb

In [ ]:
wandb.login()

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
loss_fn = BatchHardTripletMarginLoss(margin=0.5)
optimizer = AdamW(params_list, weight_decay=WEIGHT_DECAY)
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=WARMUP_STEPS, num_training_steps=T_MAX)

trainer = UnimodalPlaceRecognitionTrainer(
    checkpoints_dir='/kaggle/working/checkpoints',
    model=model,
    loss_fn=loss_fn,
    wandb_log=True,
    optimizer=optimizer,
    scheduler=scheduler,
    batch_expansion_threshold=0.7,
    device='cuda',
)

In [ ]:
wandb.init()

wandb: Currently logged in as: a-zamyshevskaya (mine_nimble). Use `wandb login --relogin` to force relogin


In [ ]:
trainer.train(
    epochs=8,
    train_dataloader=train_dl,
    val_dataloader=val_dl,
    test_dataloader=test_dl,
)

2024-07-03 19:18:22.545 | INFO     | opr.trainers.place_recognition.unimodal:train:113 - =====> Epoch:   1/8:
2024-07-03 19:18:22.547 | INFO     | opr.trainers.place_recognition.unimodal:_loop_epoch:244 - => Train stage:
2024-07-03 19:21:52.031 | INFO     | opr.trainers.place_recognition.unimodal:_loop_epoch:277 - Train time: 03:29
2024-07-03 19:21:52.033 | INFO     | opr.trainers.place_recognition.unimodal:_loop_epoch:278 - Train stats: {'loss': 1.3753778770052154, 'avg_embedding_norm': 10.919632045702002, 'num_triplets': 16.0, 'num_non_zero_triplets': 15.068965517241379, 'non_zero_rate': 0.9418103448275862, 'max_pos_pair_dist': 6.459713601517952, 'max_neg_pair_dist': 5.483606398790732, 'mean_pos_pair_dist': 5.3001554532982835, 'mean_neg_pair_dist': 4.592523936567636, 'min_pos_pair_dist': 4.1654526502236555, 'min_neg_pair_dist': 3.9849961680927497}
2024-07-03 19:21:52.034 | INFO     | opr.trainers.place_recognition.unimodal:_loop_epoch:244 - => Val stage:
2024-07-03 19:22:50.582 | INF

In [ ]:
trainer.test(test_dl, distance_threshold=5)

2024-07-03 20:11:40.051 | INFO     | opr.trainers.place_recognition.unimodal:test:172 - => Test stage:
2024-07-03 20:14:06.466 | DEBUG    | opr.trainers.place_recognition.unimodal:test:194 - Test embeddings: (1855, 1536)
2024-07-03 20:14:15.030 | INFO     | opr.trainers.place_recognition.unimodal:test:228 - Test time: 02:34
2024-07-03 20:14:15.032 | INFO     | opr.trainers.place_recognition.unimodal:test:229 - Mean Recall@N:
[0.75562399 0.88127643 0.91546123 0.93768096 0.94994068 0.95799139
 0.96267286 0.96626773 0.97016465 0.97266847 0.97376773 0.97459808
 0.9778952  0.97927836 0.98010075 0.9817496  0.98285498 0.98423545
 0.98534307 0.98563242 0.98619044 0.98646502 0.98673959 0.98729194
 0.9878393 ]
2024-07-03 20:14:15.033 | INFO     | opr.trainers.place_recognition.unimodal:test:230 - Mean Recall@1% = 0.9579913918811037
2024-07-03 20:14:15.035 | INFO     | opr.trainers.place_recognition.unimodal:test:231 - Mean top-1 distance = 3.716408165799979


In [ ]:
embeddings = extract_embeddings(model, descriptor_key="final_descriptor", dataloader=test_dl, device='cuda')
test_submission(embeddings, dataset_df=test_dl.dataset.dataset_df, filename="tuned_baseline_submission.txt")

Calculating metrics: 100%|██████████| 6/6 [00:08<00:00,  1.40s/it]


In [ ]:
torch.save(model.state_dict(), 'model.pt')

На основе статьи модификация пайплайна сабмита.

In [ ]:
import itertools
import pandas as pd
from sklearn.neighbors import KDTree
import numpy as np
import torch
from tqdm import tqdm

def extract_embeddings(model, descriptor_key, dataloader, device):
    model = model.to(device)
    model.eval()
    with torch.no_grad():
        test_embeddings_list = []
        for data in tqdm(dataloader, desc="Calculating test set descriptors"):
            batch = data
            batch = {e: batch[e].to(device) for e in batch}
            batch_embeddings = model(batch)
            test_embeddings_list.append(batch_embeddings[descriptor_key].cpu().numpy())
        test_embeddings = np.vstack(test_embeddings_list)
    return test_embeddings

def extract_patches(emb, patch_size, stride):
    if len(emb.shape) == 1:
        # патчи с помощью скользящего окна
        patches = []
        for i in range(0, len(emb) - patch_size + 1, stride):
            patch = emb[i:i + patch_size]
            patches.append(patch)
        return np.array(patches)


    patches = []
    for i in range(0, emb.shape[0] - patch_size + 1, stride):
        for j in range(0, emb.shape[1] - patch_size + 1, stride):
            patch = emb[i:i + patch_size, j:j + patch_size]
            patches.append(patch.flatten())
    return np.array(patches)

def re_rank_candidates(query_embs, database_embs, top_k=20, patch_size=64, stride=32):
    database_tree = KDTree(database_embs)
    distances, indices = database_tree.query(query_embs, k=top_k)
    re_ranked_indices = np.zeros((len(query_embs), top_k), dtype=int)

    for i in range(len(query_embs)):
        initial_candidates = indices[i]
        candidate_embs = database_embs[initial_candidates]
        query_emb = query_embs[i]

        query_patches = extract_patches(query_emb, patch_size, stride)
        candidate_patches = [extract_patches(candidate_emb, patch_size, stride) for candidate_emb in candidate_embs]

        new_distances = []
        for candidate_patch_embs in candidate_patches:
            patch_distances = np.linalg.norm(query_patches[:, np.newaxis] - candidate_patch_embs, axis=2)
            new_distances.append(np.mean(patch_distances))

        new_distances = np.array(new_distances)
        re_ranked_order = np.argsort(new_distances)
        re_ranked_indices[i] = initial_candidates[re_ranked_order]

    return re_ranked_indices

def test_submission(
    test_embeddings: np.ndarray, dataset_df: pd.DataFrame, filename: str = "submission.txt", re_rank=True
) -> None:
    """Function to create submission txt file.

    Args:
        test_embeddings (np.ndarray): Array of embeddings.
        dataset_df (pd.Dataframe): Test dataset dataframe ('test.csv').
        filename (str): Name of the output txt file. Defaults to "submission.txt".
        re_rank (bool): Whether to re-rank the candidates. Defaults to False.
    """
    tracks = []

    for _, group in dataset_df.groupby("track"):
        tracks.append(group.index.to_numpy())
    n = 3
    ij_permutations = sorted(list(itertools.permutations(range(len(tracks)), 2)))
    # ij_permutations = [(0, 1), (0, 2), (1, 0), (1, 2), (2, 0), (2, 1)]

    submission_lines = []

    for i, j in tqdm(ij_permutations, desc="Calculating metrics"):
        query_indices = tracks[i]
        database_indices = tracks[j]
        query_embs = test_embeddings[query_indices]
        database_embs = test_embeddings[database_indices]

        if re_rank:
            top_k_indices = re_rank_candidates(query_embs, database_embs, top_k=n)
            best_indices = top_k_indices[:, 0]
        else:
            database_tree = KDTree(database_embs)
            _, indices = database_tree.query(query_embs, k=1)
            best_indices = indices.squeeze()

        submission_lines.extend(list(database_indices[best_indices]))

    with open(filename, "w") as f:
        for l in submission_lines:
            f.write(str(l)+"\n")

In [ ]:
embeddings = extract_embeddings(model, descriptor_key="final_descriptor", dataloader=test_dl, device='cuda')

Calculating test set descriptors: 100%|██████████| 8/8 [02:27<00:00, 18.47s/it]


In [ ]:
test_submission(embeddings, dataset_df=test_dl.dataset.dataset_df, filename="new_baseline_submission.txt")

Calculating metrics: 100%|██████████| 6/6 [00:12<00:00,  2.05s/it]
